## MonarQ's transpiler

MonarQ's transpiler is typically used behind the scene when you use the "monarq.default" device

In [1]:
from pennylane_snowflurry.API.client import MonarqClient
from pennylane_snowflurry.transpiler.config import MonarqDefaultConfig
from pennylane_snowflurry.transpiler.steps import ISMAGS
import pennylane as qml

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
client = MonarqClient("your host", "your user", "your access token", "your project")

# no configuration argument => config is set to MonarqDefaultConfig
dev = qml.device("monarq.default", shots=1000, client=client) 

@qml.qnode(dev)
def circuit():
    qml.Hadamard(0)
    qml.CNOT([0, 1])
    qml.CNOT([1, 2])
    return qml.counts(wires=[0, 1, 2])

results = circuit()
print(results)

{'000': 357, '001': 37, '010': 71, '011': 57, '100': 77, '101': 36, '110': 99, '111': 266}


The usual behaviour of the transpiler is the following : 
1. decompose the circuit to the clifford + T gate set
2. map the circuit to the machine's topology using a pathfinding heuristic
3. route the unconnected 2 qubit gates using swaps
4. optimize by decomposing iteratively to RX, RZ and CZ, commuting, merging and cancelling inverses and trivial gates
5. decomposing to MonarQ's native gate set

It is possible to change the behaviour of the transpiler by changing steps, or adding new ones

In [ ]:
from pennylane_snowflurry.transpiler.steps import ReadoutErrorMitigation

my_config = MonarqDefaultConfig() # this is the usual behaviour

ismags_placement = ISMAGS() # this is an alternative placement method that uses subgraph isomorphism

my_config.steps[1] = ismags_placement # the placement step is 2nd. change it for the new one
my_config.steps.append(ReadoutErrorMitigation())
dev = qml.device("monarq.default", shots=1000, client=client, behaviour_config=my_config) # set the config explicitely

You can also use the transpiler in a standalone manner if need be. 

In [8]:
from pennylane_snowflurry.transpiler import PreProcessor
from pennylane.tape import QuantumTape

In [9]:
# create a QuantumTape (this is one way to do it, but you can also retrieve it from a QNode)

ops = [qml.Hadamard(0), qml.CNOT([0, 1])]
measurements = [qml.counts(wires=[0, 1])]
shots = 1000

tape = QuantumTape(ops=ops, measurements=measurements, shots=shots)

tape.operations


[H(0), CNOT(wires=[0, 1])]

In [10]:
# configure and run the transpiler

config = MonarqDefaultConfig(use_benchmark=False)
transpiler = PreProcessor.get_processor(config, [0, 1])

# since the transpiler function is a transform, you have to index [0][0] in order to get the resulting tape
tape = transpiler(tape)[0][0]

tape.operations

[X90(wires=[0]), X90(wires=[4]), Z(4), CZ(wires=[0, 4]), X90(wires=[4])]